In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import shutil
import os
import subprocess
import threading

def check_and_activate_conda_env():
    try:
        # Check if the 'manga' environment is active
        env_name = subprocess.run('echo %CONDA_DEFAULT_ENV%', shell=True, capture_output=True, text=True)
        
        if 'manga' not in env_name.stdout.strip():
            # If not active, activate the environment
            subprocess.run('conda activate manga', shell=True, check=True, executable="/bin/bash")
    except subprocess.CalledProcessError:
        messagebox.showerror("Error", "Failed to activate the Conda environment 'manga'.")
        root.quit()


def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    
    if not file_path:
        return

    demo_dir = os.path.join(os.getcwd(), 'demo')
    if not os.path.exists(demo_dir):
        os.makedirs(demo_dir)

    image_name = os.path.basename(file_path)
    demo_image_path = os.path.join(demo_dir, image_name)
    
    shutil.copy(file_path, demo_image_path)

    # Load and display the input image
    img = Image.open(demo_image_path)
    img = img.resize((400, 400))  # Resize the image to fit in the window
    img_tk = ImageTk.PhotoImage(img)
    input_image_display.config(image=img_tk)
    input_image_display.image = img_tk

    # Start the loading animation
    stop_loading_animation.clear()
    animate_loading()

    # Run the inference command in a separate thread
    threading.Thread(target=run_inference, args=(demo_image_path, image_name)).start()

def run_inference(image_path, image_name):
    try:
        subprocess.run(['python', 'inference.py', '-p', image_path], check=True)
        
        # Generate the output image path
        demo_dir = os.path.join(os.getcwd(), 'demo')
        output_image_path = os.path.join(demo_dir, f"{os.path.splitext(image_name)[0]}_colorized.png")
        
        # Load the output image
        output_image = Image.open(output_image_path)
        output_image = output_image.resize((400, 400))  # Resize the image to fit in the window
        output_image_tk = ImageTk.PhotoImage(output_image)
        
        # Update the UI
        root.after(0, lambda: show_result(output_image_tk))
    except subprocess.CalledProcessError:
        root.after(0, lambda: messagebox.showerror("Error", "Failed to run the colorization script."))
    finally:
        # Stop the loading animation
        stop_loading_animation.set()

def show_result(image_tk):
    loading_label.config(text="")  # Clear the loading text
    output_image_display.config(image=image_tk)
    output_image_display.image = image_tk
    messagebox.showinfo("Success", "Image colorization completed.")

def animate_loading():
    if stop_loading_animation.is_set():
        return
    
    # Rotate through loading symbols
    current_symbol = loading_symbols[animate_loading.index]
    loading_label.config(text=current_symbol)
    animate_loading.index = (animate_loading.index + 1) % len(loading_symbols)
    root.after(200, animate_loading)  # Update every 200 milliseconds

animate_loading.index = 0
loading_symbols = ["|", "/", "-", "\\"]  # Symbols for loading animation
stop_loading_animation = threading.Event()

# Set up the main Tkinter window
root = tk.Tk()
root.title("Manga Colorization")
root.geometry("900x700")  # Increase window size to accommodate both images and loading animation

# Create a frame for the image display area
image_frame = ttk.Frame(root)
image_frame.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

# Create labels and images inside the image_frame
input_image_label = ttk.Label(image_frame, text="Input Image")
input_image_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)

output_image_label = ttk.Label(image_frame, text="Output Image")
output_image_label.grid(row=0, column=1, padx=10, pady=5, sticky=tk.E)

input_image_display = ttk.Label(image_frame)
input_image_display.grid(row=1, column=0, padx=10, pady=5)

output_image_display = ttk.Label(image_frame)
output_image_display.grid(row=1, column=1, padx=10, pady=5)

# Create a label for the loading animation
loading_label = ttk.Label(root, text="", font=("Courier", 24))
loading_label.pack(pady=10)

# Create a button to upload the image and place it at the bottom
upload_btn = ttk.Button(root, text="Upload Gray Image", command=upload_image)
upload_btn.pack(side=tk.BOTTOM, pady=20)

# Run the Tkinter event loop
root.mainloop()


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import shutil
import os
import subprocess
import threading
from PIL import Image, ImageTk

def upload_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    
    if not file_path:
        return

    demo_dir = os.path.join(os.getcwd(), 'demo')
    if not os.path.exists(demo_dir):
        os.makedirs(demo_dir)

    image_name = os.path.basename(file_path)
    demo_image_path = os.path.join(demo_dir, image_name)
    
    shutil.copy(file_path, demo_image_path)
    
    # Start the loading animation in a new thread
    loading_thread = threading.Thread(target=animate_loading)
    loading_thread.start()

    # Run the inference command in a separate thread
    inference_thread = threading.Thread(target=run_inference, args=(demo_image_path, loading_thread))
    inference_thread.start()

def run_inference(image_path, loading_thread):
    try:
        subprocess.run(['python', 'inference.py', '-p', image_path], check=True)
        messagebox.showinfo("Success", "Image colorization completed.")
    except subprocess.CalledProcessError:
        messagebox.showerror("Error", "Failed to run the colorization script.")
    finally:
        # Stop the loading animation
        stop_loading_animation.set()
        loading_thread.join()

def animate_loading():
    stop_loading_animation.clear()
    while not stop_loading_animation.is_set():
        loading_label.config(image=loading_gif)
        root.update_idletasks()

# Set up the main Tkinter window
root = tk.Tk()
root.title("Manga Colorization")
root.geometry("800x600")  # Set the window size to 800x600 pixels

# Load the GIF
loading_gif = ImageTk.PhotoImage(file="Eclipse@1x-1.0s-200px-200px.gif")
loading_label = ttk.Label(root)
loading_label.pack(pady=20)
stop_loading_animation = threading.Event()

# Create a button to upload the image
upload_btn = ttk.Button(root, text="Upload Gray Image", command=upload_image)
upload_btn.pack(pady=20)

# Run the Tkinter event loop
root.mainloop()
